## Overview
N'hésitez pas à faire ce TP sous google collab pour des calculs plus rapides

Le transfert de style, c'est :
- prendre une image source :
<img src="https://github.com/tensorflow/models/blob/master/research/nst_blogpost/Green_Sea_Turtle_grazing_seagrass.jpg?raw=1" alt="Drawing" style="width: 100px;"/>


- prendre une image avec un certain 'style' :
<img src="https://github.com/tensorflow/models/blob/master/research/nst_blogpost/The_Great_Wave_off_Kanagawa.jpg?raw=1" alt="Drawing" style="width: 100px;"/>


- transformer l'image source en lui appliquant le 'style' de la seconde image :
<img src="https://github.com/tensorflow/models/blob/master/research/nst_blogpost/wave_turtle.png?raw=1" alt="Drawing" style="width: 100px;"/>

Dans ce TP, nous allons   : 
- définir deux notions deux distance entre images, une distance de "style" $L_{style}$, et une distance de "contenu" $L_{content}$.
- puis trouver (générer) l'image qui minimise au mieux la distance de "contenu" avec l'image source, tout en minimisant la distance de "style" avec l'image de style.


# Import et chargement des images

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import vgg19
import os

os.makedirs("./out", exist_ok=True)
content_path = './Green_Sea_Turtle_grazing_seagrass.jpg'
style_path = './The_Great_Wave_off_Kanagawa.jpg'

# Dimensions of the generated picture.
width, height = keras.preprocessing.image.load_img(content_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

- Comme nous allons appliquer vgg19 pour obtenir des features de l'image, il va nous falloir appliquer le preprocessing qu'il attend pour que l'entrée soit bien de la bonne forme :

In [2]:
def preprocess_image(image_path):
    # Util function to open, resize and format pictures into appropriate tensors
    img = keras.preprocessing.image.load_img(
        image_path, target_size=(img_nrows, img_ncols)
    )
    img = keras.preprocessing.image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return tf.convert_to_tensor(img)

- Pour visualiser l'image que nous allons générer, il nous faudra faire l'inverse de tout le preprocessing de vgg19, pour que celle-ci se retrouve bien au format que sait afficher.
    - on redonne la forme normale d'une image (nb_line, nb_colonne, 3)
    - on de-normalize
    - on ramene en rgb
    - on ramene les pixel en uint8

In [3]:
def deprocess_image(x):
    # Util function to convert a tensor into a valid image
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype("uint8")
    return x

# 2- Les metrique de distance

Comment faire une différence de contenu et de style ? 
La réponse est proche de ce que nous avons vu en transfert learning:
- les valeurs des dernieres couches de convolutions, qui contiennent des concepts plus complexes, seront similaires sur des images ayant le même contenu
- le "style" d'une image peut être retrouvé par la présence d'éléments (couleurs, coups de pinceau, variations de pixels) souvent présent simultanéments sur une images. Les valeurs des couches proches de l'entrée et celles intermedaire auront des combinaisons similaires sur des images de mêmes styles

<img src="https://miro.medium.com/max/724/0*E6BE6GDv-53smX0B.jpg" alt="Drawing" style="width: 500px;"/>

In [4]:
vgg = tf.keras.applications.vgg19.VGG19(include_top=False, weights='imagenet')
vgg.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [5]:
# List of layers to use for the style loss.
style_layer_names = [
    "block1_conv1",
    "block2_conv1",
    "block3_conv1",
    "block4_conv1",
    "block5_conv1",
]
# The layer to use for the content loss.
content_layer_name = "block5_conv2"

- la distance de contenu peut-être simplement définie comme la difference pixel à pixel entre deux image
    - tf.square(A-B) permet de mesure la différence (au carré) pixel a pixel entre A et B
    - tf.reduce_sum permet d'obtenir la difference totale, i.e. la somme des différence entre chaque pixels de A et B 

In [6]:
def content_loss(base, combination):
    return tf.reduce_sum(tf.square(combination - base))

- la matrice de gram est une matrice permettant de mesurer les correlations entre les features calculées par une couche
    - c = dimension des channels (features)
    - h*w = toutes les valeurs dans les différentes zones de l'image que peut prendre la feature

<img src="https://datascience-enthusiast.com/figures/NST_GM.png" alt="gram" style="width: 500px;"/>
source : https://datascience-enthusiast.com/DL/Art_Generation_with_Neural_Style_Transfer_v2.html

In [7]:
def gram_matrix(x):
    x = tf.transpose(x, (2, 0, 1))
    features = tf.reshape(x, (tf.shape(x)[0], -1)) # concatene les valeurs que peuvent prendre chacune des features
    gram = tf.matmul(features, tf.transpose(features))
    return gram

- La loss de style est ainsi la différence entre les matrixes de gram de style et combination calculée comme la précédente
    - calculer cette loss comme la loss précédente appliquée sur les matrices de gram de style et combination
    - entre pratique, elle est normalisée en divisant par un facteur dépendant de la taille de l'entrée :
       
            (4.0 * (channels ** 2) * (img_nrows * img_ncols ** 2))

In [8]:
width, height = keras.preprocessing.image.load_img(content_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)


def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return tf.reduce_sum(tf.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))


- Pour que ça marche mieux, on a besoin d'une troisieme loss, dont le but est de garder l'image générée localement cohérente (avec en moyenne peu de variation entre deux pixels proches)
    - elle mesure les différences entre pixels adjacent sur la même ligne (a) et adjacents sur la même colonne colonne (b)
    - puis calcule la somme de cette erreur

In [9]:
def total_variation_loss(x):
    a = tf.square(x[:, :img_nrows-1, :img_ncols-1, :] - x[:, 1:, :img_ncols-1, :])
    b = tf.square(x[:, :img_nrows-1, :img_ncols-1, :] - x[:, :img_nrows -1, 1:, :])
    return tf.reduce_sum(tf.pow(a + b, 1.25))

# Créer et entrainer notre "modele"

- charger la partie extraction de feature de vgg19 comme nous l'avions fait dans le tp transfert learning
    - rappel : weights="imagenet", include_top=False

In [10]:
model = vgg19.VGG19(weights="imagenet", include_top=False)

- Recupere le nom et sortie de tout les layers de vgg19, et crée un modèle les retournant tous, que l'on puisse calculer notre loss à partir de nimporte quelle couche

In [11]:
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
feature_extractor = keras.Model(inputs=model.inputs, outputs=outputs_dict)

- notre fonction de loss finale :
    - batches les 3 images en entrée
    - applique une extraction des feature de vgg sur ces 3 images
        - ça va plus vite que de le faire 1 par 1
    - recupere les valeurs des features map necessaires et calcule la loss totales en additionnant celles-ci

In [12]:
# Pondération des différentes loss
total_variation_weight = 1e-6
style_weight = 1e-6
content_weight = 2.5e-8

def compute_loss(combination_image, base_image, style_reference_image):
    input_tensor = tf.concat(
        [base_image, style_reference_image, combination_image], axis=0
    )
    features = feature_extractor(input_tensor)

    # Initialize the loss
    loss = tf.zeros(shape=())

    # Ajoute la content loss
    layer_features = features[content_layer_name]
    base_image_features = layer_features[0, :, :, :]
    combination_features = layer_features[2, :, :, :]
    loss += content_weight * content_loss(
        base_image_features, combination_features
    )
    
    # Ajoute la style loss : somme de la loss de style pour chaqun 
    for layer_name in style_layer_names:
        layer_features = features[layer_name]
        style_reference_features = layer_features[1, :, :, :]
        combination_features = layer_features[2, :, :, :]
        sl = style_loss(style_reference_features, combination_features)
        loss += (style_weight / len(style_layer_names)) * sl

    # Add la total variation loss
    loss += total_variation_weight * total_variation_loss(combination_image)
    return loss

- calcule les gradients de l'erreur

In [13]:
@tf.function
def compute_loss_and_grads(combination_image, base_image, style_reference_image):
    with tf.GradientTape() as tape:
        loss = compute_loss(combination_image, base_image, style_reference_image)
    grads = tape.gradient(loss, combination_image)
    return loss, grads

- choisi l'optimiser, qui va nous servir à pour minimiser ces gradients.
    - mes parametres : SGD avec exponential decay, i.e diminution du taux d'apprentissage au cours du temps : initial_learning_rate=100.0, decay_steps=100, decay_rate=0.96
        - keras.optimizers.SGD(keras.optimizers.schedules.ExponentialDecay(...))
    - le learning rate peut être très fort car on travaille que sur une seule "couche" : l'image que l'on veut générer

In [14]:
optimizer = keras.optimizers.SGD(
    keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=100.0, decay_steps=100, decay_rate=0.96
    )
)

- chargeons notre image de contenu, image de style et initialisons l'image que nous allons générée
    - pour accelerer l'entrainement, l'image que nous allons générée est initialisée à partir de l'image de contenu

In [15]:
base_image = preprocess_image(content_path)
style_reference_image = preprocess_image(style_path)
combination_image = tf.Variable(preprocess_image(content_path))

# On est enfin parti !

- On est parti !
- Les images intermediaires sont sauvegardées dans le dossier "./out", que vous puissiez visualiser l'avancement

In [ ]:
iterations = 1000
for i in range(1, iterations + 1):
    loss, grads = compute_loss_and_grads(
        combination_image, base_image, style_reference_image
    )
    # calcule la nouvelle valeur de notre image générée a partir des gradients de l'erreur, pour que celle-ci mene à une erreur plus petite
    optimizer.apply_gradients([(grads, combination_image)])
    if i % 1 == 0:
        print("Iteration %d: loss=%.2f" % (i, loss))
        img = deprocess_image(combination_image.numpy())
        fname = "_at_iteration_%d.png" % i
        keras.preprocessing.image.save_img(os.path.join("./out",fname), img)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Iteration 1: loss=160100.20
Iteration 2: loss=123223.02
Iteration 3: loss=78976.93
Iteration 4: loss=59035.00
Iteration 5: loss=48621.62
Iteration 6: loss=43871.79
Iteration 7: loss=43129.62
Iteration 8: loss=39021.86
Iteration 9: loss=38081.81
Iteration 10: loss=32747.04
Iteration 11: loss=30923.53
Iteration 12: loss=28485.93
Iteration 13: loss=27032.87
Iteration 14: loss=25661.67
Iteration 15: l

Iteration 264: loss=8073.48
Iteration 265: loss=8066.55
Iteration 266: loss=8059.66
Iteration 267: loss=8052.81
Iteration 268: loss=8046.01
Iteration 269: loss=8039.25
Iteration 270: loss=8032.54
Iteration 271: loss=8025.86
Iteration 272: loss=8019.22
Iteration 273: loss=8012.63
Iteration 274: loss=8006.09
Iteration 275: loss=7999.57
Iteration 276: loss=7993.11
Iteration 277: loss=7986.68
Iteration 278: loss=7980.30
Iteration 279: loss=7973.94
Iteration 280: loss=7967.63
Iteration 281: loss=7961.37
Iteration 282: loss=7955.12
Iteration 283: loss=7948.92
Iteration 284: loss=7942.75
Iteration 285: loss=7936.62
Iteration 286: loss=7930.54
Iteration 287: loss=7924.48
Iteration 288: loss=7918.45
Iteration 289: loss=7912.46
Iteration 290: loss=7906.51
Iteration 291: loss=7900.58
Iteration 292: loss=7894.69
Iteration 293: loss=7888.83
Iteration 294: loss=7883.00
Iteration 295: loss=7877.21
Iteration 296: loss=7871.45
Iteration 297: loss=7865.72
Iteration 298: loss=7860.02
Iteration 299: loss=

# Pour aller plus loin

- modifier le ratio entre style et contenu pour obtenir une image plus fidele au contenu ou au style
- testez avec vos propres images !